In [777]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.datasets import load_boston
from xgboost import XGBClassifier
from statsmodels.formula.api import ols
import statsmodels.api as sm
import scipy.stats as stats
import pickle
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

from sklearn.utils import resample
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from catboost import CatBoostRegressor
from sklearn.model_selection import learning_curve
import warnings
warnings.filterwarnings('ignore')

In [778]:
df = pd.read_csv('FTTH-DataSet.xlsx - FTTH.csv')

In [779]:
df.isnull().sum()

ID                                    0
GOVERNORATE                         323
Customer with orange_MONTHS           0
CUSTOMER_AGE_MONTHS                   0
CUSTOMER_GENDER                      54
COMMITMENT                            0
COMMITMENT_FG                         0
OF_SPEED                              0
OF_PREV_SPEED                     74714
MIGRATION_FLAG                        0
LAST_LINK_QUALITY                    28
LAST_LINK_STATUS                     28
LAST_POWER_VALIDATION                28
LAST_LINK_PRIORITY                   28
Disconnection_TOTAL_MAX_day         625
Disconnection_TOTAL_MIN_day         625
Disconnection_TOTAL_SUM_Month       625
Disconnection_TOTAL_MEAN_Month      625
GB_TOTAL_CONSUMPTION_Month1         155
GB_TOTAL_CONSUMPTION_Month2         367
GB_TOTAL_CONSUMPTION_Month3        4242
TARGET                                0
dtype: int64

In [780]:
df['median_GB'] = df[['GB_TOTAL_CONSUMPTION_Month1', 'GB_TOTAL_CONSUMPTION_Month2', 'GB_TOTAL_CONSUMPTION_Month3']].median(axis=1)
df['CUSTOMER_AGE_YEARS'] = df['CUSTOMER_AGE_MONTHS'] / 12
pd.DataFrame({'GOVERNORATE': ['EAST AMMAN', 'WEST AMMAN', 'Balqa', 'Zarqa', 'Aqaba','Irbid','Jarash','Karak','Madaba','Ma\'an']})

# replace values with "Others"
df.loc[df['GOVERNORATE'].isin(['Balqa', 'Zarqa', 'Aqaba','Irbid','Jarash','Karak','Madaba','Ma\'an']), 'GOVERNORATE'] = 'Others'


In [781]:
df = df.drop(['ID', 'OF_PREV_SPEED','CUSTOMER_AGE_MONTHS','GB_TOTAL_CONSUMPTION_Month1','GB_TOTAL_CONSUMPTION_Month2','GB_TOTAL_CONSUMPTION_Month3','LAST_LINK_QUALITY', 'LAST_LINK_STATUS', 'LAST_POWER_VALIDATION', 'LAST_LINK_PRIORITY', 'Disconnection_TOTAL_MAX_day', 'Disconnection_TOTAL_MIN_day', 'Disconnection_TOTAL_SUM_Month', 'Disconnection_TOTAL_MEAN_Month'], axis=1)

In [782]:
df

,GOVERNORATE,Customer with orange_MONTHS,CUSTOMER_GENDER,COMMITMENT,COMMITMENT_FG,OF_SPEED,MIGRATION_FLAG,TARGET,median_GB,CUSTOMER_AGE_YEARS
0,West Amman,49,M,24,1,200,y,0,562.0,47.333333
1,West Amman,45,M,24,0,100,y,0,160.0,61.750000
2,West Amman,45,M,24,1,200,y,0,299.0,44.250000
3,Others,44,M,24,0,200,y,0,569.0,53.833333
4,West Amman,42,M,12,1,400,y,0,891.0,26.000000
...,...,...,...,...,...,...,...,...,...,...
94473,Others,1,M,12,1,200,y,0,315.0,20.083333
94474,East Amman,18,M,24,1,200,n,0,0.0,42.166667
94475,West Amman,13,F,12,1,400,y,0,0.0,71.750000
94476,West Amman,9,M,24,1,200,n,0,NaN,27.833333


In [783]:
df =df.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [784]:
df.isnull().sum()

GOVERNORATE                    0
Customer with orange_MONTHS    0
CUSTOMER_GENDER                0
COMMITMENT                     0
COMMITMENT_FG                  0
OF_SPEED                       0
MIGRATION_FLAG                 0
TARGET                         0
median_GB                      0
CUSTOMER_AGE_YEARS             0
dtype: int64

In [785]:
df

,GOVERNORATE,Customer with orange_MONTHS,CUSTOMER_GENDER,COMMITMENT,COMMITMENT_FG,OF_SPEED,MIGRATION_FLAG,TARGET,median_GB,CUSTOMER_AGE_YEARS
0,West Amman,49,M,24,1,200,y,0,562.0,47.333333
1,West Amman,45,M,24,0,100,y,0,160.0,61.750000
2,West Amman,45,M,24,1,200,y,0,299.0,44.250000
3,Others,44,M,24,0,200,y,0,569.0,53.833333
4,West Amman,42,M,12,1,400,y,0,891.0,26.000000
...,...,...,...,...,...,...,...,...,...,...
94473,Others,1,M,12,1,200,y,0,315.0,20.083333
94474,East Amman,18,M,24,1,200,n,0,0.0,42.166667
94475,West Amman,13,F,12,1,400,y,0,0.0,71.750000
94476,West Amman,9,M,24,1,200,n,0,0.0,27.833333


In [786]:
def median_GB(df):
     # Round the 'median_GB' column and convert to integers
    df['median_GB'] = df['median_GB'].round().astype(int)
    return df
df=median_GB(df)

In [787]:
def CUSTOMER_AGE_YEARS(df):
     # Round the 'CUSTOMER_AGE_YEARS' column and convert to integers
    df['CUSTOMER_AGE_YEARS'] = df['CUSTOMER_AGE_YEARS'].round().astype(int)
    return df
df=CUSTOMER_AGE_YEARS(df)

In [788]:
df.median_GB

0        562
1        160
2        299
3        569
4        891
        ... 
94473    315
94474      0
94475      0
94476      0
94477      0
Name: median_GB, Length: 94478, dtype: int32

In [789]:
def Encoding(df):
 label_encoder = preprocessing.LabelEncoder()
 df['GOVERNORATE']= label_encoder.fit_transform(df['GOVERNORATE'])
 df['CUSTOMER_GENDER']= label_encoder.fit_transform(df['CUSTOMER_GENDER'])
 df['MIGRATION_FLAG']= label_encoder.fit_transform(df['MIGRATION_FLAG'])
 columns_to_scale = ['GOVERNORATE','median_GB', 'Customer with orange_MONTHS', 'CUSTOMER_AGE_YEARS','CUSTOMER_GENDER', 'COMMITMENT', 'COMMITMENT_FG', 'OF_SPEED','MIGRATION_FLAG', 'TARGET']
 scaler = MinMaxScaler()
 df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
 #Separate the features (X) and target variable (y)
 X = df.drop('TARGET', axis=1)
 y = df['TARGET']
 return(df)
df=Encoding(df)

In [790]:
# Define function to remove outliers
def remove_outliers(df):
    # List of variables to check for outliers
    variables = ['GOVERNORATE','Customer with orange_MONTHS', 'CUSTOMER_GENDER', 'COMMITMENT','COMMITMENT_FG',  
                 'OF_SPEED', 'MIGRATION_FLAG', 'TARGET', 'median_GB','CUSTOMER_AGE_YEARS']
    # Iterate over the variables and remove outliers
    for variable in variables:
        df = df[np.abs(df[variable]-df[variable].mean()) <= (3*df[variable].std())]
        
    return df
df=remove_outliers(df)

In [791]:
def imbalance (df):
 # Define oversampler
 oversample = SMOTE()

 # Apply oversampling on the data
 X_oversampled, y_oversampled = oversample.fit_resample(X, y)
 return(df)

In [792]:
X = df[['GOVERNORATE', 'Customer with orange_MONTHS', 'CUSTOMER_GENDER', 'COMMITMENT', 'COMMITMENT_FG', 'OF_SPEED', 'MIGRATION_FLAG', 'median_GB', 'CUSTOMER_AGE_YEARS']]
y = df['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [793]:
df.to_csv('ORANGE_FIBER_FIXED.csv')

In [794]:
## model trai 1 (Linear regression)

In [795]:
#DEVELOPMENT

In [796]:
# Create a Linear regressor
lm = LinearRegression()

# Train the model using the training data
lm.fit(X_train, y_train)

# Model prediction on train data
y_pred = lm.predict(X_train)
# Predicting Test data with the model
y_test_pred = lm.predict(X_test)
# Model metrics
acc_lm = metrics.r2_score(y_test, y_test_pred)
RMSE1T=np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

In [797]:
#EVALUATION

In [798]:
def summary_statistics(model, X_train, y_train, X_test, y_test):

    print(f"Train set score: {model.score(X_train, y_train)}")
    print(f"Test set score: {model.score(X_test, y_test)}")
    print(f"Adjusted R^2: {1 - (1 - metrics.r2_score(y_train, y_pred)) * (len(y_train) - 1) / (len(y_train) - X_train.shape[1] - 1)}")

In [799]:
summary_statistics(lm, X_train, y_train, X_test, y_test)
print('R^2:', acc_lm)
print('RMSE:',RMSE1T)

Train set score: 1.0
Test set score: 1.0
Adjusted R^2: 1.0
R^2: 1.0
RMSE: 0.0


In [800]:
##model 2 (Decision Tree Regressor)

In [801]:
#Development

In [802]:
# Create a Decision Tree regressor
clf = tree.DecisionTreeRegressor(min_samples_split=13, max_depth=10, min_samples_leaf=10, max_leaf_nodes=320)

# Train the model using the training data
clf.fit(X_train, y_train)

# Predict the test data
y_pred = clf.predict(X_train)
# Predicting Test data with the model
y_test_pred = clf.predict(X_test)
# Model Metrics
acc_clf = metrics.r2_score(y_test, y_test_pred)
RMSE2T = np.sqrt(metrics.mean_squared_error(y_train, y_pred))

In [803]:
#Evaluation

In [804]:
summary_statistics(clf, X_train, y_train, X_test, y_test)
print('R^2:', acc_clf)
print('RMSE:',RMSE2T)

Train set score: 1.0
Test set score: 1.0
Adjusted R^2: 1.0
R^2: 1.0
RMSE: 0.0


In [805]:
##model 3 (Random Forest model)

In [806]:
#Development

In [807]:
# Create a Random Forest Regressor
reg = RandomForestRegressor()

# Train the model using the training sets 
reg.fit(X_train, y_train)

# Fit the model to the training data with different hyperparameters
reg = RandomForestRegressor(n_estimators= 110, min_samples_split= 13, max_depth= 9) 
reg.fit(X_train, y_train)

# Model prediction on train data
y_pred = reg.predict(X_train)
# Predicting Test data with the model
y_test_pred = reg.predict(X_test)
# Model metrics
acc_reg = metrics.r2_score(y_test, y_test_pred)
RMSE3T = np.sqrt(metrics.mean_squared_error(y_train, y_pred))

In [808]:
#Evaluation

In [809]:
summary_statistics(reg, X_train, y_train, X_test, y_test)
print('R^2:', acc_reg)
print('RMSE:',RMSE3T)

Train set score: 1.0
Test set score: 1.0
Adjusted R^2: 1.0
R^2: 1.0
RMSE: 0.0


In [810]:
# Save model to a pickle file
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X, y)

# Save the trained model using pickle
with open("Orange_Fiber.pkl", "wb") as f:
    pickle.dump(rf, f)

In [811]:
##SUMMARY

In [812]:
models = pd.DataFrame({
    'Model': ['Linear Regression', 'Decision Tree' ,'Random Forest'], 'RMSE' : [RMSE1T,RMSE2T,RMSE3T], 
    'R2 Score': [acc_lm*100, acc_clf*100,acc_reg*100]})
models.sort_values(by='R2 Score', ascending=False)

,Model,RMSE,R2 Score
0,Linear Regression,0.0,100.0
1,Decision Tree,0.0,100.0
2,Random Forest,0.0,100.0


In [813]:
##we will use RandomForest